In [3]:
import json

response = {
  "id": "msg_01RpFGeHxxSNcG43VMwbVGMF",
  "type": "message",
  "role": "assistant",
  "model": "claude-opus-4-5-20251101",
  "content": [
    {
      "type": "text",
      "text": "# 📘 README.md - Prompt Engineering Lab with Gemini API\n\n## 🎯 ภาพรวมของ Lab (Lab Overview)\n\nLab นี้ออกแบบมาเพื่อให้นักศึกษาได้เรียนรู้และฝึกฝนเทคนิค **Prompt Engineering** ขั้นสูงผ่าน **Google Gemini API** โดยครอบคลุมตั้งแต่พื้นฐานไปจนถึงเทคนิคขั้นสูง พร้อมระบบประเมินผล (Evaluation) ที่ครบถ้วน\n\n---\n\n## 📚 สารบัญ (Table of Contents)\n\n1. [ทฤษฎีพื้นฐาน Prompt Engineering](#-ทฤษฎีพื้นฐาน-prompt-engineering)\n2. [สถาปัตยกรรมของ Lab](#-สถาปัตยกรรมของ-lab)\n3. [เทคนิคทั้ง 8 แบบ](#-เทคนิคทั้ง-8-แบบ)\n4. [ระบบ Evaluation](#-ระบบ-evaluation)\n5. [การติดตั้งและเริ่มต้นใช้งาน](#-การติดตั้งและเริ่มต้นใช้งาน)\n6. [แบบฝึกหัดและ Challenge](#-แบบฝึกหัดและ-challenge)\n7. [แหล่งเรียนรู้เพิ่มเติม](#-แหล่งเรียนรู้เพิ่มเติม)\n\n---\n\n## 🧠 ทฤษฎีพื้นฐาน Prompt Engineering\n\n### Prompt Engineering คืออะไร?\n\n**Prompt Engineering** คือศาสตร์และศิลป์ในการออกแบบ \"คำสั่ง\" (Prompt) ที่ส่งให้ Large Language Models (LLMs) เพื่อให้ได้ผลลัพธ์ที่ต้องการอย่างแม่นยำและมีคุณภาพ\n\n```\n┌─────────────────────────────────────────────────────────────────┐\n│                    PROMPT ENGINEERING OVERVIEW                   │\n├─────────────────────────────────────────────────────────────────┤\n│                                                                  │\n│    ┌──────────┐      ┌──────────┐      ┌──────────────┐        │\n│    │  INPUT   │ ──▶  │   LLM    │ ──▶  │   OUTPUT     │        │\n│    │ (Prompt) │      │ (Gemini) │      │  (Response)  │        │\n│    └──────────┘      └──────────┘      └──────────────┘        │\n│         │                                      │                │\n│         │    ┌─────────────────────────┐      │                │\n│         └───▶│  Prompt Engineering     │◀─────┘                │\n│              │  • เทคนิคการเขียน        │                       │\n│              │  • การจัดโครงสร้าง        │                       │\n│              │  • การให้ตัวอย่าง         │                       │\n│              │  • การประเมินผล          │                       │\n│              └─────────────────────────┘                        │\n│                                                                  │\n└─────────────────────────────────────────────────────────────────┘\n```\n\n### ทำไม Prompt Engineering ถึงสำคัญ?\n\n```\n┌────────────────────────────────────────────────────────────────────┐\n│                    ความสำคัญของ Prompt Engineering                   │\n├────────────────────────────────────────────────────────────────────┤\n│                                                                     │\n│  ❌ Bad Prompt                        ✅ Good Prompt                │\n│  ┌─────────────────────┐              ┌─────────────────────┐      │\n│  │ \"สรุปให้หน่อย\"       │              │ \"สรุปบทความนี้ใน    │      │\n│  │                     │              │  3 bullet points    │      │\n│  │                     │              │  โดยเน้นประเด็นหลัก  │      │\n│  └─────────────────────┘              └─────────────────────┘      │\n│           │                                    │                    │\n│           ▼                                    ▼                    │\n│  ┌─────────────────────┐              ┌─────────────────────┐      │\n│  │ ผลลัพธ์: คลุมเครือ    │              │ ผลลัพธ์: ชัดเจน      │      │\n│  │ ไม่ตรงประเด็น        │              │ ตรงความต้องการ       │      │\n│  │ Format ไม่แน่นอน    │              │ Format เป็นระบบ     │      │\n│  └─────────────────────┘              └─────────────────────┘      │\n│                                                                     │\n│  💡 Prompt ที่ดี = ผลลัพธ์ที่ดี + ประหยัดเวลา + ลด Token Cost       │\n│                                                                     │\n└────────────────────────────────────────────────────────────────────┘\n```\n\n---\n\n## 🏗 สถาปัตยกรรมของ Lab\n\n### โครงสร้าง Lab ทั้งหมด\n\n```\n┌─────────────────────────────────────────────────────────────────────────┐\n│                        LAB ARCHITECTURE                                  │\n├─────────────────────────────────────────────────────────────────────────┤\n│                                                                          │\n│   PART 0: Setup                                                          │\n│   ════════════════                                                       │\n│   ┌─────────────────────────────────────────────────────────────┐       │\n│   │  • ติดตั้ง google-genai library                              │       │\n│   │  • กำหนด API Key และ Model                                  │       │\n│   │  • ทดสอบ Connection                                         │       │\n│   └─────────────────────────────────────────────────────────────┘       │\n│                              │                                           │\n│                              ▼                                           │\n│   PART 1-8: เทคนิค Prompt Engineering                                   │\n│   ════════════════════════════════════                                   │\n│   ┌──────────────┬──────────────┬──────────────┬──────────────┐        │\n│   │   Part 1     │   Part 2     │   Part 3     │   Part 4     │        │\n│   │  Few-shot    │    CoT       │Self-Consist. │Prompt Chain  │        │\n│   │     ⭐       │    ⭐⭐      │   ⭐⭐⭐     │   ⭐⭐⭐     │        │\n│   └──────────────┴──────────────┴──────────────┴──────────────┘        │\n│   ┌──────────────┬──────────────┬──────────────┬──────────────┐        │\n│   │   Part 5     │   Part 6     │   Part 7     │   Part 8     │        │\n│   │ Reflection   │    ToT       │Self-Feedback │Self-Critique │        │\n│   │   ⭐⭐⭐⭐   │  ⭐⭐⭐⭐⭐  │   ⭐⭐⭐⭐   │  ⭐⭐⭐⭐⭐  │        │\n│   └──────────────┴──────────────┴──────────────┴──────────────┘        │\n│                              │                                           │\n│                              ▼                                           │\n│   PART 9-10: Evaluation                                                  │\n│   ════════════════════════                                               │\n│   ┌─────────────────────────────────────────────────────────────┐       │\n│   │  • LLM-as-a-Judge       • Pairwise Comparison               │       │\n│   │  • Reference-free       • Comprehensive Evaluation          │       │\n│   └─────────────────────────────────────────────────────────────┘       │\n│                              │                                           │\n│                              ▼                                           │\n│   PART 11: Final Challenge                                               │\n│   ════════════════════════                                               │\n│   ┌─────────────────────────────────────────────────────────────┐       │\n│   │         รวมทุกเทคนิคเข้าด้วยกัน (Combined Techniques)         │       │\n│   └─────────────────────────────────────────────────────────────┘       │\n│                                                                          │\n└─────────────────────────────────────────────────────────────────────────┘\n```\n\n---\n\n## 🔧 เทคนิคทั้ง 8 แบบ\n\n### 1. Few-shot Prompting ⭐\n\n**ทฤษฎี:** การให้ตัวอย่าง (examples) ใน prompt เพื่อช่วยให้ LLM เข้าใจรูปแบบที่ต้องการ\n\n```\n┌─────────────────────────────────────────────────────────────────┐\n│                      FEW-SHOT PROMPTING                          │\n├─────────────────────────────────────────────────────────────────┤\n│                                                                  │\n│   ┌─────────────┐   ┌─────────────┐   ┌─────────────┐          │\n│   │  Zero-shot  │   │  One-shot   │   │  Few-shot   │          │\n│   │  (0 ตัวอย่าง) │   │  (1 ตัวอย่าง) │   │ (2-5 ตัวอย่าง)│          │\n│   └──────┬──────┘   └──────┬──────┘   └──────┬──────┘          │\n│          │                 │                 │                   │\n│          ▼                 ▼                 ▼                   │\n│   ┌─────────────┐   ┌─────────────┐   ┌─────────────┐          │\n│   │ ความแม่นยำ   │   │ ความแม่นยำ   │   │ ความแม่นยำ   │          │\n│   │    ต่ำ       │   │    กลาง     │   │    สูง      │          │\n│   └─────────────┘   └─────────────┘   └─────────────┘          │\n│                                                                  │\n│   📝 ตัวอย่างการใช้งาน:                                          │\n│   ┌─────────────────────────────────────────────────────────┐   │\n│   │ ตัวอย่าง 1: \"อร่อยมาก!\" → บวก                           │   │\n│   │ ตัวอย่าง 2: \"ผิดหวัง\" → ลบ                               │   │\n│   │ ตัวอย่าง 3: \"ก็โอเค\" → กลาง                              │   │\n│   │ ─────────────────────────────────                       │   │\n│   │ Input: \"อาหารดี แต่รอนาน\" → ???                          │   │\n│   └─────────────────────────────────────────────────────────┘   │\n│                                                                  │\n│   ✅ เหมาะสำหรับ: Data Transformation, Classification,          │\n│                   Format Standardization                        │\n│                                                                  │\n└─────────────────────────────────────────────────────────────────┘\n```\n\n**เมื่อไหร่ควรใช้:**\n- ต้องการ output format เฉพาะ\n- Task ที่ต้องการความสม่ำเสมอ\n- การแปลงข้อมูลตาม pattern\n\n---\n\n### 2. Chain-of-Thought (CoT) ⭐⭐\n\n**ทฤษฎี:** กระตุ้นให้ LLM แสดงขั้นตอนการคิดอย่างเป็นลำดับก่อนให้คำตอบสุดท้าย\n\n```\n┌─────────────────────────────────────────────────────────────────┐\n│                    CHAIN-OF-THOUGHT (CoT)                        │\n├─────────────────────────────────────────────────────────────────┤\n│                                                                  │\n│   ❌ Without CoT                    ✅ With CoT                  │\n│   ┌─────────────────┐              ┌─────────────────┐          │\n│   │    คำถาม        │              │    คำถาม        │          │\n│   │       │         │              │       │         │          │\n│   │       ▼         │              │       ▼         │          │\n│   │    คำตอบ        │              │   ขั้นตอน 1     │          │\n│   │  (อาจผิดพลาด)    │              │       │         │          │\n│   │                 │              │       ▼         │          │\n│   │                 │              │   ขั้นตอน 2     │          │\n│   │                 │              │       │         │          │\n│   │                 │              │       ▼         │          │\n│   │                 │              │   ขั้นตอน 3     │          │\n│   │                 │              │       │         │          │\n│   │                 │              │       ▼         │          │\n│   │                 │              │    คำตอบ        │          │\n│   │                 │              │   (แม่นยำกว่า)   │          │\n│   └─────────────────┘              └─────────────────┘          │\n│                                                                  │\n│   💡 Trigger Phrases:                                            │\n│   ┌─────────────────────────────────────────────────────────┐   │\n│   │  • \"Let's think step by step\"                           │   │\n│   │  • \"คิดทีละขั้นตอน\"                                       │   │\n│   │  • \"แสดงวิธีคิด\"                                          │   │\n│   │  • \"อธิบายเหตุผลของคุณ\"                                    │   │\n│   └─────────────────────────────────────────────────────────┘   │\n│                                                                  │\n│   ✅ เหมาะสำหรับ: Math Problems, Logical Reasoning,             │\n│                   Complex Analysis, Decision Making             │\n│                                                                  │\n└─────────────────────────────────────────────────────────────────┘\n```\n\n**ตัวอย่างผลลัพธ์:**\n```\nโจทย์: ร้านค้าขายเสื้อ 3 ตัว ราคาตัวละ 250 บาท ลด 10% จ่ายเท่าไหร่?\n\nCoT Response:\nขั้นตอน 1: คำนวณราคาเสื้อทั้งหมด = 3 × 250 = 750 บาท\nขั้นตอน 2: คำนวณส่วนลด = 750 × 10% = 75 บาท\nขั้นตอน 3: ราคาสุทธิ = 750 - 75 = 675 บาท\nคำตอบ: 675 บาท\n```\n\n---\n\n### 3. Self-Consistency CoT ⭐⭐⭐\n\n**ทฤษฎี:** สร้างคำตอบหลายครั้งด้วยเส้นทางการคิดที่แตกต่างกัน แล้วเลือกคำตอบที่ consistent มากที่สุด\n\n```\n┌─────────────────────────────────────────────────────────────────┐\n│                   SELF-CONSISTENCY CoT                           │\n├─────────────────────────────────────────────────────────────────┤\n│                                                                  │\n│                         ┌─────────┐                             │\n│                         │ คำถาม   │                             │\n│                         └────┬────┘                             │\n│                              │                                   │\n│              ┌───────────────┼───────────────┐                  │\n│              ▼               ▼               ▼                  │\n│       ┌─────────────┐ ┌─────────────┐ ┌─────────────┐          │\n│       │  Path 1     │ │  Path 2     │ │  Path 3     │          │\n│       │ (T=0.7)     │ │ (T=0.7)     │ │ (T=0.7)     │          │\n│       │             │ │             │ │             │          │\n│       │ คำตอบ: 42   │ │ คำตอบ: 42   │ │ คำตอบ: 45   │          │\n│       └──────┬──────┘ └──────┬──────┘ └──────┬──────┘          │\n│              │               │               │                  │\n│              └───────────────┼───────────────┘                  │\n│                              ▼                                   │\n│                    ┌─────────────────┐                          │\n│                    │  Majority Vote  │                          │\n│                    │  42: 2 votes    │                          │\n│                    │  45: 1 vote     │                          │\n│                    └────────┬────────┘                          │\n│                              │                                   │\n│                              ▼                                   │\n│                    ┌─────────────────┐                          │\n│                    │  Final: 42 ✅   │                          │\n│                    │  (Confidence:   │                          │\n│                    │   66.7%)        │                          │\n│                    └─────────────────┘                          │\n│                                                                  │\n│   📊 Voting Methods:                                             │\n│   • Majority Voting: เลือกคำตอบที่มีมากที่สุด                     │\n│   • Weighted Voting: ถ่วงน้ำหนักด้วย confidence                  │\n│                                                                  │\n│   ✅ เหมาะสำหรับ: Complex Problems, High-stakes Decisions,       │\n│                   Problems needing verification                 │\n│                                                                  │\n└─────────────────────────────────────────────────────────────────┘\n```\n\n---\n\n### 4. Prompt Chaining ⭐⭐⭐\n\n**ทฤษฎี:** แบ่งงานซับซ้อนออกเป็นขั้นตอนย่อย โดย output ของ prompt หนึ่งจะเป็น input ของ prompt ถัดไป\n\n```\n┌─────────────────────────────────────────────────────────────────┐\n│                      PROMPT CHAINING                             │\n├─────────────────────────────────────────────────────────────────┤\n│                                                                  │\n│   ┌─────────────────────────────────────────────────────────┐   │\n│   │                    Complex Task                          │   │\n│   │               \"วิเคราะห์บทความธุรกิจ\"                       │   │\n│   └─────────────────────────────────────────────────────────┘   │\n│                              │                                   │\n│                              ▼                                   │\n│   ╔═══════════════════════════════════════════════════════════╗ │\n│   ║  Chain 1: Extract                                          ║ │\n│   ║  ┌─────────────────────────────────────────────────────┐  ║ │\n│   ║  │ • สกัดข้อมูลสำคัญ                                     │  ║ │\n│   ║  │ • ระบุตัวเลข/สถิติ                                    │  ║ │\n│   ║  │ • หาบุคคล/องค์กรที่เกี่ยวข้อง                           │  ║ │\n│   ║  └──────────────────────────┬──────────────────────────┘  ║ │\n│   ╚═════════════════════════════╪═════════════════════════════╝ │\n│                                 │ Output 1                      │\n│                                 ▼                                │\n│   ╔═══════════════════════════════════════════════════════════╗ │\n│   ║  Chain 2: Analyze                                          ║ │\n│   ║  ┌─────────────────────────────────────────────────────┐  ║ │\n│   ║  │ Input: Output 1                                      │  ║ │\n│   ║  │ • วิเคราะห์ Sentiment                                 │  ║ │\n│   ║  │ • ระบุ Tone                                          │  ║ │\n│   ║  │ • หากลุ่มเป้าหมาย                                      │  ║ │\n│   ║  └──────────────────────────┬──────────────────────────┘  ║ │\n│   ╚═════════════════════════════╪═════════════════════════════╝ │\n│                                 │ Output 2                      │\n│                                 ▼                                │\n│   ╔═══════════════════════════════════════════════════════════╗ │\n│   ║  Chain 3: Synthesize                                       ║ │\n│   ║  ┌─────────────────────────────────────────────────────┐  ║ │\n│   ║  │ Input: Output 1 + Output 2                           │  ║ │\n│   ║  │ • สร้างสรุป                                           │  ║ │\n│   ║  │ • ให้ข้อเสนอแนะ                                        │  ║ │\n│   ║  │ • เสนอแนวทาง                                          │  ║ │\n│   ║  └─────────────────────────────────────────────────────┘  ║ │\n│   ╚═══════════════════════════════════════════════════════════╝ │\n│                                 │                                │\n│                                 ▼                                │\n│   ┌─────────────────────────────────────────────────────────┐   │\n│   │                    Final Output                          │   │\n│   │           บทวิเคราะห์ที่สมบูรณ์และเป็นระบบ                    │   │\n│   └─────────────────────────────────────────────────────────┘   │\n│                                                                  │\n│   ✅ เหมาะสำหรับ: Multi-step Analysis, Document Processing,      │\n│                   Complex Workflows                             │\n│                                                                  │\n└─────────────────────────────────────────────────────────────────┘\n```\n\n---\n\n### 5. Reflection ⭐⭐⭐⭐\n\n**ทฤษฎี:** ให้ LLM ทบทวนและประเมินคำตอบของตัวเอง แล้วปรับปรุงให้ดีขึ้น\n\n```\n┌─────────────────────────────────────────────────────────────────┐\n│                        REFLECTION                                │\n├─────────────────────────────────────────────────────────────────┤\n│                                                                  │\n│   ┌─────────────────────────────────────────────────────────┐   │\n│   │  Phase 1: GENERATE                                       │   │\n│   │  ┌─────────────────────────────────────────────────┐    │   │\n│   │  │          สร้างคำตอบเบื้องต้น                        │    │   │\n│   │  │              (Initial Response)                   │    │   │\n│   │  └─────────────────────────────────────────────────┘    │   │\n│   └─────────────────────────────────────────────────────────┘   │\n│                              │                                   │\n│                              ▼                                   │\n│   ┌─────────────────────────────────────────────────────────┐   │\n│   │  Phase 2: REFLECT                                        │   │\n│   │  ┌─────────────────────────────────────────────────┐    │   │\n│   │  │  🔍 วิเคราะห์คำตอบ:                                │    │   │\n│   │  │  • จุดแข็ง (Strengths)                            │    │   │\n│   │  │  • จุดอ่อน (Weaknesses)                           │    │   │\n│   │  │  • สิ่งที่ขาดหาย (Missing)                         │    │   │\n│   │  │  • ข้อผิดพลาด (Errors)                            │    │   │\n│   │  └─────────────────────────────────────────────────┘    │   │\n│   └─────────────────────────────────────────────────────────┘   │\n│                              │                                   │\n│                              ▼                                   │\n│   ┌─────────────────────────────────────────────────────────┐   │\n│   │  Phase 3: IMPROVE                                        │   │\n│   │  ┌─────────────────────────────────────────────────┐    │   │\n│   │  │          ✨ สร้างคำตอบที่ปรับปรุงแล้ว                 │    │   │\n│   │  │              (Improved Response)                  │    │   │\n│   │  │                                                   │    │   │\n│   │  │  แก้ไข: จุดอ่อน ✓ สิ่งที่ขาด ✓ ข้อผิดพลาด ✓        │    │   │\n│   │  └─────────────────────────────────────────────────┘    │   │\n│   └─────────────────────────────────────────────────────────┘   │\n│                                                                  │\n│   🔄 Iterative Reflection: สามารถทำซ้ำได้หลายรอบจนพอใจ          │\n│                                                                  │\n│   ✅ เหมาะสำหรับ: Content Writing, Code Generation,              │\n│                   Creative Tasks                                │\n│                                                                  │\n└─────────────────────────────────────────────────────────────────┘\n```\n\n---\n\n### 6. Tree-of-Thought (ToT) ⭐⭐⭐⭐⭐\n\n**ทฤษฎี:** สำรวจหลายเส้นทางการคิดพร้อมกัน เหมือนการเล่นหมากรุกที่พิจารณาหลายตาล่วงหน้า\n\n```\n┌─────────────────────────────────────────────────────────────────┐\n│                     TREE-OF-THOUGHT (ToT)                        │\n├─────────────────────────────────────────────────────────────────┤\n│                                                                  │\n│                         ┌─────────┐                             │\n│                         │ Problem │                             │\n│                         └────┬────┘                             │\n│                              │                                   │\n│            ┌─────────────────┼─────────────────┐                │\n│            ▼                 ▼                 ▼                │\n│     ┌────────────┐    ┌────────────┐    ┌────────────┐         │\n│     │ Approach A │    │ Approach B │    │ Approach C │         │\n│     │ Score: 7/10│    │ Score: 8/10│    │ Score: 5/10│         │\n│     └─────┬──────┘    └─────┬──────┘    └─────┬──────┘         │\n│           │                 │                 │                 │\n│           │           ┌─────┴─────┐           │                 │\n│           │           ▼           ▼           │                 │\n│           │     ┌─────────┐ ┌─────────┐       │                 │\n│           │     │ B.1     │ │ B.2     │       │                 │\n│           │     │Score:9/10│Score:7/10│       │                 │\n│           │     └────┬────┘ └─────────┘       │                 │\n│           │          │                        │                 │\n│           │          ▼                        │                 │\n│           │    ┌──────────┐                   │                 │\n│           │    │ B.1 ✅    │                   │                 │\n│           │    │ SELECTED │                   │                 │\n│           │    │ SOLUTION │                   │                 │\n│           │    └──────────┘                   │                 │\n│           │                                   │                 │\n│      ❌ Pruned                            ❌ Pruned             │\n│                                                                  │\n│   🌳 Process:                                                    │\n│   1. Generate: สร้างหลายทางเลือก                                 │\n│   2. Evaluate: ประเมินแต่ละทาง                                   │\n│   3. Expand: ขยายทางที่ดี                                        │\n│   4. Select: เลือกทางที่ดีที่สุด                                  │\n│                                                                  │\n│   ✅ เหมาะสำหรับ: Strategic Planning, Complex Problem Solving,   │\n│                   Multi-criteria Decisions                      │\n│                                                                  │\n└─────────────────────────────────────────────────────────────────┘\n```\n\n---\n\n### 7. Self-Feedback ⭐⭐⭐⭐\n\n**ทฤษฎี:** ให้ LLM สร้าง feedback ให้ตัวเองตามเกณฑ์ที่กำหนด แล้วใช้ feedback นั้นปรับปรุงคำตอบ\n\n```\n┌─────────────────────────────────────────────────────────────────┐\n│                       SELF-FEEDBACK                              │\n├─────────────────────────────────────────────────────────────────┤\n│                                                                  │\n│   ┌─────────────────────────────────────────────────────────┐   │\n│   │                    CRITERIA                              │   │\n│   │  ┌─────────┐ ┌─────────┐ ┌─────────┐ ┌─────────┐       │   │\n│   │  │Accuracy │ │Clarity  │ │Complete │ │Engaging │       │   │\n│   │  │ ness    │ │         │ │ ness    │ │         │       │   │\n│   │  └─────────┘ └─────────┘ └─────────┘ └─────────┘       │   │\n│   └─────────────────────────────────────────────────────────┘   │\n│                              │                                   │\n│                              ▼                                   │\n│   ┌─────────────────────────────────────────────────────────┐   │\n│   │  STEP 1: Generate Response                               │   │\n│   │  ────────────────────────────────────────────────────   │   │\n│   │  \"AI กำลังเปลี่ยนแปลงตลาดแรงงานไทย...\"                     │   │\n│   └─────────────────────────────────────────────────────────┘   │\n│                              │                                   │\n│                              ▼                                   │\n│   ┌─────────────────────────────────────────────────────────┐   │\n│   │  STEP 2: Self-Feedback                                   │   │\n│   │  ────────────────────────────────────────────────────   │   │\n│   │  📊 Accuracy:    7/10 - \"ข้อมูลถูกต้องแต่ขาดแหล่งอ้างอิง\"   │   │\n│   │  📊 Clarity:     8/10 - \"อ่านเข้าใจง่าย\"                   │   │\n│   │  📊 Completeness:6/10 - \"ขาดตัวอย่างที่เป็นรูปธรรม\"        │   │\n│   │  📊 Engaging:    7/10 - \"ควรเพิ่ม hook ที่ดึงดูด\"          │   │\n│   └─────────────────────────────────────────────────────────┘   │\n│                              │                                   │\n│                              ▼                                   │\n│   ┌─────────────────────────────────────────────────────────┐   │\n│   │  STEP 3: Improve based on Feedback                       │   │\n│   │  ────────────────────────────────────────────────────   │   │\n│   │  ✨ \"ในปี 2024 AI ได้สร้างผลกระทบต่อตลาดแรงงานไทย         │   │\n│   │     อย่างมีนัยสำคัญ ตามรายงานของ...\"                       │   │\n│   └─────────────────────────────────────────────────────────┘   │\n│                                                                  │\n│   🔑 ความแตกต่างจาก Reflection:                                  │\n│   • Reflection: ทบทวนทั่วไป                                     │\n│   • Self-Feedback: ใช้เกณฑ์เฉพาะในการประเมิน                    │\n│                                                                  │\n│   ✅ เหมาะสำหรับ: Content Creation, Quality Assurance,           │\n│                   Structured Improvement                        │\n│                                                                  │\n└─────────────────────────────────────────────────────────────────┘\n```\n\n---\n\n### 8. Self-Critique ⭐⭐⭐⭐⭐\n\n**ทฤษฎี:** LLM วิพากษ์คำตอบของตัวเองอย่างเข้มงวด พยายามหาข้อผิดพลาด จุดอ่อน และสมมติฐานที่ผิดพลาด\n\n```\n┌─────────────────────────────────────────────────────────────────┐\n│                      SELF-CRITIQUE                               │\n├─────────────────────────────────────────────────────────────────┤\n│                                                                  │\n│   ┌─────────────────────────────────────────────────────────┐   │\n│   │                   ADVERSARIAL MODE 🎯                    │   │\n│   │         \"เป็นนักวิจารณ์ที่เข้มงวด - อย่าประนีประนอม!\"         │   │\n│   └─────────────────────────────────────────────────────────┘   │\n│                                                                  │\n│   ┌─────────────┐                                               │\n│   │  Response   │                                               │\n│   └──────┬──────┘                                               │\n│          │                                                       │\n│          ▼                                                       │\n│   ╔═══════════════════════════════════════════════════════════╗ │\n│   ║  🔍 CRITIQUE CHECKLIST                                     ║ │\n│   ╠═══════════════════════════════════════════════════════════╣ │\n│   ║  □ Logical Fallacies (ข้อผิดพลาดทางตรรกะ)                  ║ │\n│   ║  □ Unsupported Assumptions (สมมติฐานไม่มีหลักฐาน)           ║ │\n│   ║  □ Incorrect Information (ข้อมูลที่อาจไม่ถูกต้อง)            ║ │\n│   ║  □ Weak Arguments (การโต้แย้งที่อ่อน)                       ║ │\n│   ║  □ Missing Elements (สิ่งที่ขาดหาย)                         ║ │\n│   ║  □ Potential Biases (อคติที่อาจมี)                          ║ │\n│   ╚═══════════════════════════════════════════════════════════╝ │\n│          │                                                       │\n│          ▼                                                       │\n│   ┌─────────────────────────────────────────────────────────┐   │\n│   │  Round 1 Critique:                                       │   │\n│   │  ⚠️ \"สมมติฐานที่ว่า AI จะแทนที่งานทั้งหมดไม่มีหลักฐาน...\"    │   │\n│   └──────────────────────────┬──────────────────────────────┘   │\n│                              │                                   │\n│                              ▼                                   │\n│   ┌─────────────────────────────────────────────────────────┐   │\n│   │  Improved Response (Round 2)                             │   │\n│   └──────────────────────────┬──────────────────────────────┘   │\n│                              │                                   │\n│                              ▼                                   │\n│   ┌─────────────────────────────────────────────────────────┐   │\n│   │  Round 2 Critique:                                       │   │\n│   │  ⚠️ \"ยังขาดมุมมองจากฝั่งลูกจ้าง...\"                        │   │\n│   └──────────────────────────┬──────────────────────────────┘   │\n│                              │                                   │\n│                              ▼                                   │\n│   ┌─────────────────────────────────────────────────────────┐   │\n│   │  ✅ Final Synthesized Response                           │   │\n│   │  (รวมจุดแข็ง หลีกเลี่ยงจุดอ่อน สมดุลและรอบคอบ)               │   │\n│   └─────────────────────────────────────────────────────────┘   │\n│                                                                  │\n│   🔑 ความแตกต่างจาก Self-Feedback:                               │\n│   • Self-Feedback: ประเมินตามเกณฑ์ที่กำหนด                      │\n│   • Self-Critique: วิพากษ์อย่างเข้มงวด หา \"ข้อผิดพลาด\"          │\n│                                                                  │\n│   ✅ เหมาะสำหรับ: Controversial Topics, Important Decisions,     │\n│                   Arguments that need to be bulletproof         │\n│                                                                  │\n└─────────────────────────────────────────────────────────────────┘\n```\n\n---\n\n## 📊 ระบบ Evaluation\n\n### Overview ของ Evaluation Methods\n\n```\n┌─────────────────────────────────────────────────────────────────────────┐\n│                      EVALUATION METHODS OVERVIEW                         │\n├─────────────────────────────────────────────────────────────────────────┤\n│                                                                          │\n│   ┌─────────────────────────────────────────────────────────────────┐   │\n│   │                    1. LLM-as-a-Judge                             │   │\n│   │   ─────────────────────────────────────────────────────────────  │   │\n│   │   ใช้ LLM เป็นผู้ตัดสินคุณภาพของ response                         │   │\n│   │                                                                   │   │\n│   │   📋 Rubric:                                                     │   │\n│   │   ┌───────────────┬────────────────────────────────────────┐    │   │\n│   │   │ Relevance     │ คำตอบตรงประเด็นกับคำถามมากน้อยเพียงใด    │    │   │\n│   │   │ Accuracy      │ ข้อมูลในคำตอบถูกต้องมากน้อยเพียงใด       │    │   │\n│   │   │ Completeness  │ คำตอบครบถ้วนสมบูรณ์มากน้อยเพียงใด        │    │   │\n│   │   │ Clarity       │ คำตอบชัดเจน เข้าใจง่ายมากน้อยเพียงใด     │    │   │\n│   │   │ Helpfulness   │ คำตอบมีประโยชน์ต่อผู้ถามมากน้อยเพียงใด   │    │   │\n│   │   └───────────────┴────────────────────────────────────────┘    │   │\n│   └─────────────────────────────────────────────────────────────────┘   │\n│                                                                          │\n│   ┌─────────────────────────────────────────────────────────────────┐   │\n│   │                  2. Pairwise Comparison                          │   │\n│   │   ─────────────────────────────────────────────────────────────  │   │\n│   │   เปรียบเทียบ 2 คำตอบและเลือกอันที่ดีกว่า                         │   │\n│   │                                                                   │   │\n│   │   ┌─────────────┐     VS     ┌─────────────┐                    │   │\n│   │   │ Response A  │◀──────────▶│ Response B  │                    │   │\n│   │   └──────┬──────┘            └──────┬──────┘                    │   │\n│   │          │                          │                            │   │\n│   │          └──────────┬───────────────┘                            │   │\n│   │                     ▼                                            │   │\n│   │          ┌─────────────────┐                                     │   │\n│   │          │ Winner: A/B/Tie │                                     │   │\n│   │          └─────────────────┘                                     │   │\n│   └─────────────────────────────────────────────────────────────────┘   │\n│                                                                          │\n│   ┌─────────────────────────────────────────────────────────────────┐   │\n│   │                3. Reference-free Evaluation                      │   │\n│   │   ─────────────────────────────────────────────────────────────  │   │\n│   │   ประเมินคุณภาพโดยไม่ต้องมีคำตอบอ้างอิง                           │   │\n│   │                                                                   │   │\n│   │   📊 Evaluation Aspects:                                         │   │\n│   │   • Fluency (ความลื่นไหลของภาษา)                                  │   │\n│   │   • Coherence (ความเชื่อมโยงของเนื้อหา)                            │   │\n│   │   • Informativeness (ความให้ข้อมูล)                               │   │\n│   │   • No Hallucination (ไม่มีการสร้างข้อมูลเท็จ)                      │   │\n│   │                                                                   │   │\n│   │   🔍 พร้อมตรวจจับ Potential Hallucinations                        │   │\n│   └─────────────────────────────────────────────────────────────────┘   │\n│                                                                          │\n└─────────────────────────────────────────────────────────────────────────┘\n```\n\n### Comprehensive Evaluation Pipeline\n\n```\n┌─────────────────────────────────────────────────────────────────────────┐\n│                 COMPREHENSIVE EVALUATION PIPELINE                        │\n├─────────────────────────────────────────────────────────────────────────┤\n│                                                                          │\n│   ┌────────────┐                                                        │\n│   │  Response  │                                                        │\n│   │ to Evaluate│                                                        │\n│   └─────┬──────┘                                                        │\n│         │                                                                │\n│         ▼                                                                │\n│   ┌─────────────────────────────────────────────────────────────────┐   │\n│   │                                                                  │   │\n│   │  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐          │   │\n│   │  │ LLM-as-Judge │  │Reference-free│  │ Hallucination│          │   │\n│   │  │   Score: 8   │  │   Score: 7   │  │  Risk: LOW   │          │   │\n│   │  └──────────────┘  └──────────────┘  └──────────────┘          │   │\n│   │         │                 │                 │                   │   │\n│   │         └─────────────────┼─────────────────┘                   │   │\n│   │                           ▼                                     │   │\n│   │                  ┌────────────────┐                             │   │\n│   │                  │   AGGREGATE    │                             │   │\n│   │                  │   SCORING      │                             │   │\n│   │                  └───────┬────────┘                             │   │\n│   │                          │                                      │   │\n│   │                          ▼                                      │   │\n│   │   ┌─────────────────────────────────────────────────────────┐  │   │\n│   │   │  📊 Final Report                                        │  │   │\n│   │   │  ─────────────────────────────────────────────────────  │  │   │\n│   │   │  Overall Score: 7.5/10                                  │  │   │\n│   │   │  Strengths: Clear, Accurate, Well-structured            │  │   │\n│   │   │  Weaknesses: Missing examples                           │  │   │\n│   │   │  Hallucination Risk: Low ✅                              │  │   │\n│   │   │  Recommendations: Add concrete examples                 │  │   │\n│   │   └─────────────────────────────────────────────────────────┘  │   │\n│   │                                                                  │   │\n│   └──────────────────────────────────────────────────────────────────┘   │\n│                                                                          │\n└─────────────────────────────────────────────────────────────────────────┘\n```\n\n---\n\n## 🚀 การติดตั้งและเริ่มต้นใช้งาน\n\n### Prerequisites\n\n```bash\n# ติดตั้ง Library ที่จำเป็น\npip install google-genai pandas numpy\n```\n\n### Setup\n\n```python\n# Import Libraries\nfrom google import genai\nfrom google.genai import types\n\n# กำหนด API Key และ Model\nAPI_KEY = \"YOUR_API_KEY_HERE\"\nMODEL_NAME = \"gemini-2.0-flash\"\n\n# สร้าง Client\nclient = genai.Client(api_key=API_KEY)\n\n# ทดสอบ Connection\nresponse = client.models.generate_content(\n    model=MODEL_NAME,\n    contents=\"สวัสดี! ตอบสั้นๆ ว่าคุณพร้อมทำงานแล้ว\"\n)\nprint(\"✅ Connection Test:\", response.text)\n```\n\n### การขอ API Key\n\n1. ไปที่ [Google AI Studio](https://aistudio.google.com/)\n2. สร้าง Project ใหม่\n3. ไปที่ Settings > API Keys\n4. สร้าง API Key ใหม่\n\n---\n\n## 📝 แบบฝึกหัดและ Challenge\n\n### แบบฝึกหัด 1: Few-shot Prompting\n**โจทย์:** สร้าง Few-shot prompt สำหรับแปลงชื่อผลิตภัณฑ์เป็น SKU code\n- ตัวอย่าง: \"เสื้อยืดสีขาว ไซส์ L\" → \"TSH-WHT-L\"\n- สร้างอย่างน้อย 3 ตัวอย่าง\n\n### แบบฝึกหัด 2: Chain-of-Thought\n**โจทย์:** ใช้ CoT แก้ปัญหา:\n\"บริษัทมีพนักงาน 50 คน ต้องการเพิ่มเป็น 75 คนภายใน 6 เดือน \nถ้าทุก 2 เดือนรับพนักงานใหม่จำนวนเท่าๆ กัน แต่ละรอบต้องรับกี่คน?\"\n\n### Final Challenge: Combined Techniques\n**โจทย์:** วิเคราะห์ปัญหา EdTech Startup:\n- ผู้ใช้ 10,000 คน\n- Retention rate 15%\n- เป้าหมาย: เพิ่มเป็น 40% ใน 6 เดือน\n- งบ 2 ล้านบาท\n\n---\n\n## 📋 สรุป Cheat Sheet\n\n### เทคนิคและเมื่อไหร่ควรใช้\n\n| เทคนิค | ใช้เมื่อ | ความซับซ้อน | API Calls |\n|--------|---------|------------|-----------|\n| Few-shot | ต้องการ format เฉพาะ | ⭐ | 1 |\n| Chain-of-Thought | ปัญหาต้องใช้เหตุผล | ⭐⭐ | 1 |\n| Self-Consistency | ต้องการความมั่นใจ | ⭐⭐⭐ | 3-5 |\n| Prompt Chaining | งานซับซ้อนหลายขั้นตอน | ⭐⭐⭐ | 2-5 |\n| Reflection | ต้องการปรับปรุงคำตอบ | ⭐⭐⭐⭐ | 3 |\n| Tree-of-Thought | ปัญหามีหลายทางเลือก | ⭐⭐⭐⭐⭐ | 3-5 |\n| Self-Feedback | ต้องการประเมินตามเกณฑ์ | ⭐⭐⭐⭐ | 3 |\n| Self-Critique | ต้องการหาจุดอ่อน | ⭐⭐⭐⭐⭐ | 4-6 |\n\n### Decision Flowchart\n\n```\n┌─────────────────────────────────────────────────────────────────────────┐\n│                    TECHNIQUE SELECTION FLOWCHART                         │\n├─────────────────────────────────────────────────────────────────────────┤\n│                                                                          │\n│                         ┌─────────────┐                                 │\n│                         │   START     │                                 │\n│                         └──────┬──────┘                                 │\n│                                │                                         │\n│                                ▼                                         │\n│                 ┌──────────────────────────────┐                        │\n│                 │ ต้องการ format เฉพาะหรือไม่?   │                        │\n│                 └───────────────┬──────────────┘                        │\n│                        YES │         │ NO                               │\n│                            ▼         ▼                                  │\n│                   ┌─────────────┐ ┌──────────────────────────────┐     │\n│                   │  Few-shot   │ │ ต้องใช้การคิดหลายขั้นตอนหรือไม่? │     │\n│                   └─────────────┘ └───────────────┬──────────────┘     │\n│                                          YES │         │ NO            │\n│                                              ▼         ▼               │\n│                                 ┌─────────────┐ ┌─────────────┐        │\n│                                 │    CoT      │ │ Direct Prompt│        │\n│                                 └──────┬──────┘ └─────────────┘        │\n│                                        │                                │\n│                                        ▼                                │\n│                      ┌──────────────────────────────┐                  │\n│                      │ ต้องการความมั่นใจสูงหรือไม่?    │                  │\n│                      └───────────────┬──────────────┘                  │\n│                             YES │         │ NO                         │\n│                                 ▼         │                            │\n│                    ┌───────────────────┐  │                            │\n│                    │Self-Consistency   │  │                            │\n│                    │      CoT          │  │                            │\n│                    └───────────────────┘  │                            │\n│                                           ▼                            │\n│                      ┌──────────────────────────────┐                  │\n│                      │ มีหลายทางเลือกต้องพิจารณาหรือไม่?│                  │\n│                      └───────────────┬──────────────┘                  │\n│                             YES │         │ NO                         │\n│                                 ▼         │                            │\n│                    ┌───────────────────┐  │                            │\n│                    │ Tree-of-Thought   │  │                            │\n│                    └───────────────────┘  │                            │\n│                                           ▼                            │\n│                      ┌──────────────────────────────┐                  │\n│                      │ ต้องการปรับปรุงคำตอบหรือไม่?    │                  │\n│                      └───────────────┬──────────────┘                  │\n│                             YES │         │ NO                         │\n│                                 ▼         │                            │\n│                  ┌───────────────────────┐│                            │\n│                  │ มีเกณฑ์ชัดเจนหรือไม่?   ││                            │\n│                  └─────────────┬─────────┘│                            │\n│                       YES │    │ NO       │                            │\n│                           ▼    ▼          │                            │\n│              ┌─────────────┐┌────────────┐│                            │\n│              │Self-Feedback││ Reflection ││                            │\n│              └─────────────┘└────────────┘│                            │\n│                                           ▼                            │\n│                      ┌──────────────────────────────┐                  │\n│                      │ เป็นหัวข้อถกเถียงได้หรือไม่?    │                  │\n│                      └───────────────┬──────────────┘                  │\n│                             YES │         │ NO                         │\n│                                 ▼         ▼                            │\n│                    ┌───────────────────┐ ┌─────────────┐               │\n│                    │  Self-Critique    │ │ Basic Prompt│               │\n│                    └───────────────────┘ └─────────────┘               │\n│                                                                          │\n└─────────────────────────────────────────────────────────────────────────┘\n```\n\n---\n\n## 📚 แหล่งเรียนรู้เพิ่มเติม\n\n### Documentation\n- [Google AI Documentation](https://ai.google.dev/)\n- [Gemini API Reference](https://ai.google.dev/api)\n\n### Research Papers\n- [Chain-of-Thought Paper](https://arxiv.org/abs/2201.11903) - Wei et al., 2022\n- [Tree-of-Thought Paper](https://arxiv.org/abs/2305.10601) - Yao et al., 2023\n- [Self-Consistency Paper](https://arxiv.org/abs/2203.11171) - Wang et al., 2022\n\n### Learning Resources\n- [Prompt Engineering Guide](https://www.promptingguide.ai/)\n- [Learn Prompting](https://learnprompting.org/)\n\n---\n\n## ✅ Learning Outcomes\n\nหลังจากเรียนจบ Lab นี้ นักศึกษาจะสามารถ:\n\n1. ✅ เข้าใจหลักการ Prompt Engineering และเทคนิคขั้นสูงทั้ง 8 แบบ\n2. ✅ เลือกใช้เทคนิคที่เหมาะสมกับปัญหาต่างๆ ได้อย่างถูกต้อง\n3. ✅ ประยุกต์ใช้ Gemini API ในการสร้าง solutions\n4. ✅ ประเมินคุณภาพ Output ด้วย LLM-as-a-Judge และ Reference-free methods\n5. ✅ ผสมผสานเทคนิคต่างๆ เพื่อแก้ปัญหาที่ซับซ้อน\n\n---\n\n**🎉 Happy Prompting!**\n\n*Made with ❤️ for AI Engineering Students*"
    }
  ],
 
}


In [2]:
# Simple extraction
text = response["content"][0]["text"]
print(text)

# Save to file
with open("output.txt", "w", encoding="utf-8") as f:
    f.write(text)

# 🧪 Prompt Engineering Lab with Gemini API

## 📚 Lab Overview & Learning Objectives

Lab นี้ออกแบบมาเพื่อให้ผู้เรียนได้ฝึกฝนเทคนิค Prompt Engineering ขั้นสูงผ่าน Gemini API โดยเริ่มจากพื้นฐานไปจนถึงเทคนิคขั้นสูง พร้อมระบบ Evaluation ที่ครบถ้วน

---

```python
#%% [markdown]
# # 🧪 Prompt Engineering Advanced Lab with Gemini API
# 
# ## 📋 Lab Description
# Lab นี้จะพาคุณเรียนรู้เทคนิค Prompt Engineering ขั้นสูงทั้ง 8 เทคนิค:
# 1. Few-shot Prompting
# 2. Chain-of-Thought (CoT)
# 3. Self-consistency Chain-of-Thought
# 4. Prompt Chaining
# 5. Reflection
# 6. Tree-of-Thought (ToT)
# 7. Self-feedback
# 8. Self-critique
# 
# พร้อมระบบ Evaluation:
# - LLM-as-a-Judge
# - Reference-free Evaluation
# 
# ## 🎯 Learning Outcomes
# - เข้าใจและประยุกต์ใช้เทคนิค Prompt Engineering ขั้นสูง
# - สามารถเลือกเทคนิคที่เหมาะสมกับปัญหาต่างๆ
# - ประเมินคุณภาพ Output ด้วยวิธีการที่เป็นระบบ

#%% [markdown]
# ---
# # 🔧 Part 0: Environment Setup
# ---

#%%
# ติดตั้ง Library ที่จำเป็น
# !pip install google-genai pand